##### 1) 라이브러리 설치

In [1]:
# poetry add langsmith

In [2]:
import os
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_y


In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langsmith import traceable

# .env 파일에서 환경 변수를 로드합니다.
# 이 코드는 다른 os.getenv 호출보다 먼저 실행되어야 합니다.
load_dotenv()

# LangSmith API Key 설정
# getenv의 두 번째 인자로 기본값을 설정하여, 환경 변수가 없을 때 오류를 방지할 수 있습니다.
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING", "true")  # LangSmith 활성화 (기본값 "true")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")      # API Key 불러오기
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")      # 프로젝트 이름 설정
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")      # EndPoint 설정

# LLM 모델 설정 (Groq 사용)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",       # Groq API 엔드포인트
    api_key=os.getenv("OPENAI_API_KEY"),              # Groq API 키도 환경 변수에서 로드
    model="llama3-70b-8192",                        # Groq에서 지원하는 모델명으로 수정
    temperature=0.7
)

# LangSmith로 실행 추적
@traceable(run_type="chain", name="Simple_Chain")
def ask_question(question: str):
    """지정된 질문에 대해 AI 모델에게 답변을 요청하는 함수"""

    # 개별 메시지 템플릿 정의
    system_message = SystemMessagePromptTemplate.from_template(
        "당신은 유용한 AI 비서입니다. 한국어로 간결하고 명확하게 답변해주세요."
    )
    user_message = HumanMessagePromptTemplate.from_template(
        "{question}"
    )

    # 채팅 프롬프트 템플릿 생성
    chat_prompt = ChatPromptTemplate.from_messages([
        system_message,
        user_message,
    ])
    
    # 프롬프트와 LLM을 연결하는 체인 생성
    chain = chat_prompt | llm

    # 체인 실행 및 결과 반환
    response = chain.invoke({"question": question})

    return response.content

# 테스트 실행
question = "LangGraph와 LangChain의 가장 큰 차이점은 무엇인가요?"
answer = ask_question(question)

# 결과 출력
print("\n🔹 [AI 답변]:")
print(answer)

TypeError: str expected, not NoneType